## Instructions and Description for Amberdata Amazon S3 Bucket Query

This Python notebook is designed to perform complex financial analysis on the real-time order book data for the Ethereum-USD trading pair on the Binance exchange using Amberdata's S3 buckets. Below is an overview of the structure and functionalities of the code.

### 1. Importing Essential Libraries
The code kicks off by importing the necessary Python and PySpark libraries:

- `delta.tables`: To work with Delta tables in PySpark.
- `pyspark.sql.window`: To apply window functions in Spark SQL.
- `pyspark.sql.functions`: To use a wide range of functions in Spark SQL.
- `pyspark.sql.types`: To define specific data types.

### 2. Data Ingestion and Basic Filtering
The code reads data from an S3 bucket containing DeltaLake tables. It applies basic filters to isolate data (feel free to change the filters):

- Year: 2023
- Month: August
- Day: 29
- Exchange: Binance
- Trading Pair: ETH/USDT

### 3. Data Transformation
After reading the data, it undergoes several transformations to prepare for analysis:

- Type casting: Converting timestamp to `long` data type.
- Column selection: Isolating relevant columns like price and quantity.

#### Additional Data Filtering:

- `bid_book`: Contains the data for bid orders.
- `ask_book`: Contains the data for ask orders.

### 4. Calculating Slippage and Other Metrics
The code calculates slippage, cost, and other metrics based on the real-time order book. The following metrics are computed:

- `cum_quantity`: Cumulative sum of quantities.
- `cum_cost`: Cumulative sum of cost.
- `slippage`: Calculated based on the difference between the desired and actual order amounts.
- `currency_slippage`: Slippage in terms of currency.
- `dollar_slippage`: Slippage in terms of US dollars.
- `percent_slippage`: Slippage as a percentage of the order.

### 5. Combining Bid and Ask Books
The `bid_book` and `ask_book` are then joined on the `exchangeTimestamp` field, and aggregate metrics are computed.

### 6. Resultant Order Book Analysis
Finally, the code groups the data by `exchangeTimestamp` to calculate first non-null instances of various slippage metrics for both buying and selling.

#### Additional Features:

- The code uses the Spark DataFrame's `.cache()` method to cache intermediate DataFrames for improved performance.
- Window functions are applied for partitioning and ordering data.

### Sample Code Variables
You can set your cost basis like this:

```python
cost_basis = 1_000_000
```

This sets your cost basis to 1 million USD, and slippage calculations will be based on this.

*Note: Due to the real-time nature of the data and heavy calculations, this notebook might take a considerable amount of time to run.*

In [ ]:
from delta.tables import *
from pyspark.sql.window import Window
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.types import StructType, StructField, DataType, BooleanType, NumericType, TimestampType, IntegerType, DecimalType, StringType, LongType, DateType, FloatType
from botocore.client import Config

import numpy as np
from datetime import datetime
from itertools import groupby
from io import StringIO
import pandas as pd
import seaborn as sns
from decimal import Decimal
from matplotlib.ticker import StrMethodFormatter, PercentFormatter
import matplotlib.patches as mpatches
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import math
import requests
import json

In [ ]:
# 2. Data Ingestion and Basic Filtering
trades = (
        DeltaTable
        .forPath(spark, 's3://amberdata-marketdata-deltalake/spot/order-book-snapshots/').toDF()
        .where('year = "2023" \
                AND month = "08"\
                AND day = "29"\
                AND exchange = "binance" \
                AND pair = "eth_usdt"')
        .select(f.explode('orderBookSides')).select("col.exchangeTimestamp", "col.isBid", f.explode('col.data'))
         ).cache()

In [ ]:
# 3.1 Data Transformation: Type Casting
trades = trades.withColumn('exchangeTimestamp', f.col('exchangeTimestamp').cast('long'))

# 3.2 Data Transformation: Column Selection
trades = trades\
          .select(f.col("col")\
          .getItem(0).alias("price"), f.col("col")\
          .getItem(1).alias("quantity"), "*")\
          .drop("col", 'timestamp_window')

# 3.3 Additional Data Filtering: Bid and Ask Books
bid_book = trades.filter(f.col("isBid") == True).select("*")
ask_book = trades.filter(f.col("isBid") == False).select("*")

# 3.4 Renaming Columns
bid_book = bid_book.drop("isBid")
bid_book = bid_book.withColumnRenamed('price', 'bid_price').withColumnRenamed('quantity', 'bid_quantity')
ask_book = ask_book.drop("isBid")
ask_book = ask_book.withColumnRenamed('price', 'ask_price').withColumnRenamed('quantity', 'ask_quantity')

In [ ]:
# 4.1 Preview of Bid and Ask Books
ask_book.limit(5).display()
bid_book.limit(5).display()

In [ ]:
# 5.1 Window Functions Initialization
timestampWindow = Window.partitionBy("exchangeTimestamp")

# 5.2 Defining Windows for Bid and Ask Books
bid_window = Window.partitionBy('exchangeTimestamp')\
        .orderBy(f.desc('bid_price'))\
        .rowsBetween(Window.unboundedPreceding, 0)

ask_window = Window.partitionBy('exchangeTimestamp')\
        .orderBy(f.asc('ask_price'))\
        .rowsBetween(Window.unboundedPreceding, 0)

In [ ]:
# 6.1 Slippage and Metrics Calculation
cost_basis = 1_000_000

# 6.2 Calculations for Bid Book
bid_book = bid_book\
        .withColumn("order_amount", lit(cost_basis))\
        .withColumn("sell_order_price_1", max("bid_price").over(timestampWindow))\
        .withColumn("desired_amount", col("order_amount") / col("sell_order_price_1"))\
        .withColumn("cum_quantity", sum("bid_quantity").over(bid_window))\
        .withColumn("cum_cost", sum(col("bid_quantity") * col("bid_price")).over(bid_window))\
        .withColumn("slippage", when(col('order_amount') - col('cum_cost') < 0, ((col('order_amount') - (col('cum_cost') - (col('bid_quantity') * col('bid_price')))) / col('bid_price')) + (col('cum_quantity') - col('bid_quantity')))\
            .otherwise(0))\
        .withColumn("sell_currency_slippage", when(col('slippage') != 0, (col("desired_amount") - col("slippage")) *-1 ))\
        .withColumn("sell_dollar_slippage", when(col('slippage') != 0, (col("order_amount") - (col("slippage")*col("sell_order_price_1"))) *-1))\
        .withColumn("sell_percent_slippage", when(col('slippage') != 0, ((col("desired_amount") / col("slippage")) -1 ) *-1 ))

# 6.3 Calculations for Ask Book
ask_book = ask_book\
        .withColumn("order_amount", lit(cost_basis))\
        .withColumn("buy_order_price_1", min("ask_price").over(timestampWindow))\
        .withColumn("desired_amount", col("order_amount") / col("buy_order_price_1"))\
        .withColumn("cum_quantity", sum("ask_quantity").over(ask_window))\
        .withColumn("cum_cost", sum(col("ask_quantity") * col("ask_price")).over(ask_window))\
        .withColumn("slippage", when(col('order_amount') - col('cum_cost') < 0, ((col('order_amount') - (col('cum_cost') - (col('ask_quantity') * col('ask_price')))) / col('ask_price')) + (col('cum_quantity') - col('ask_quantity')))\
            .otherwise(0))\
        .withColumn("buy_currency_slippage", when(col('slippage') != 0, col("desired_amount") - col("slippage")))\
        .withColumn("buy_dollar_slippage", when(col('slippage') != 0, col("order_amount") - (col("slippage")*col("buy_order_price_1"))))\
        .withColumn("buy_percent_slippage", when(col('slippage') != 0,(col("desired_amount") / col("slippage"))-1))

# 6.4 Join and group Bid and Ask Book
orderbook = bid_book.join(ask_book, on='exchangeTimestamp', how='inner')

orderbook = orderbook.groupby("exchangeTimestamp")\
            .agg(first(col("buy_currency_slippage"), ignorenulls=True).alias("buy_currency_slippage"),
                 first(col("buy_dollar_slippage"), ignorenulls=True).alias("buy_dollar_slippage"),
                 first(col("buy_percent_slippage"), ignorenulls=True).alias("buy_percent_slippage"),
                 first(col("sell_currency_slippage"), ignorenulls=True).alias("sell_currency_slippage"),
                 first(col("sell_dollar_slippage"), ignorenulls=True).alias("sell_dollar_slippage"),
                 first(col("sell_percent_slippage"), ignorenulls=True).alias("sell_percent_slippage"))

In [ ]:
# 7. Display Results
orderbook.limit(5).display()

In [ ]:
def create_heatmap(df, column, title, color_palette, dpi=75):
    df['hour_of_day'] = df.index.hour
    df['day_of_week'] = df.index.dayofweek
    days = ['Mon.', 'Tue.', 'Wed.', 'Thu.', 'Fri.', 'Sat.', 'Sun.']
    df['day_of_week'] = df['day_of_week'].apply(lambda x: days[x])
    pivot = df.pivot_table(index='day_of_week', columns='hour_of_day', values=column, aggfunc='mean')
    pivot = pivot.reindex(['Sun.','Mon.', 'Tue.', 'Wed.', 'Thu.', 'Fri.', 'Sat.'])

    # Create the heatmap
    sns.heatmap(pivot, annot=False, fmt='.2f', cmap = sns.color_palette(color_palette, as_cmap=True))
    # Add x and y labels
    plt.xlabel('Hour (UTC)')
    plt.ylabel('Day')
    plt.title(title, loc='left')
    plt.gcf().set_dpi(dpi)
    # Show the plot
    plt.show()
    
def plot_regression(x, y, xlabel, ylabel, title, color, size, alpha):
    sns.set_style("darkgrid")
    sns.regplot(x=x, y=y, color=color, scatter_kws={'s':size, 'alpha':alpha})
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

def plot_slippage(slippage, title):
    plt.figure(figsize=(15,7), dpi=100)
    ax = plt.gca()

    low_color = 'green'
    high_color = 'red'

    low_quantile = slippage.quantile(.25)
    high_quantile = slippage.quantile(.75)

    plt.fill_between(merged_df.index, merged_df['Close'], 0, 
                     where=(slippage <= low_quantile),
                     facecolor=low_color, alpha = .5)

    plt.fill_between(merged_df.index, merged_df['Close'], 0, 
                     where=(slippage >= high_quantile),
                     facecolor=high_color, alpha = .5)
    
    plt.plot(merged_df.index, merged_df['Close'], color='black', linewidth=1)

    ax.set_ylim(merged_df['Close'].min()*.975, merged_df['Close'].max()*1.025)

    plt.title(title)
    plt.ylabel('ETH/USDT 1h Close Price')

    low_legend = mpatches.Patch(color=low_color, label= '≤ 25th Percentile')
    high_legend = mpatches.Patch(color=high_color, label= '≥ 75th Percentile')
    plt.legend(handles=[low_legend, high_legend])

    plt.show()
    
def plot_slippage_hv(slippage, title):
    plt.figure(figsize=(15,7), dpi=100)
    ax = plt.gca()

    low_color = 'green'
    high_color = 'red'

    low_quantile = slippage.quantile(.25)
    high_quantile = slippage.quantile(.75)

    plt.fill_between(merged_df.index, merged_df['hv'], 0, 
                     where=(slippage <= low_quantile),
                     facecolor=low_color, alpha = .5)

    plt.fill_between(merged_df.index, merged_df['hv'], 0, 
                     where=(slippage >= high_quantile),
                     facecolor=high_color, alpha = .5)
    
    plt.plot(merged_df.index, merged_df['hv'], color='black', linewidth=1)

    ax.set_ylim(merged_df['hv'].min()*.975, merged_df['hv'].max()*1.025)

    plt.title(title)
    plt.ylabel('ETH/USDT 24h Hist. Volatility (C/C)')

    low_legend = mpatches.Patch(color=low_color, label= '≤ 25th Percentile')
    high_legend = mpatches.Patch(color=high_color, label= '≥ 75th Percentile')
    plt.legend(handles=[low_legend, high_legend])

    plt.show()
    
def plot_slippage_iv(slippage, title):
    plt.figure(figsize=(15,7), dpi=100)
    ax = plt.gca()

    low_color = 'green'
    high_color = 'red'

    low_quantile = slippage.quantile(.25)
    high_quantile = slippage.quantile(.75)

    plt.fill_between(merged_df.index, merged_df['atm7'], 0, 
                     where=(slippage <= low_quantile),
                     facecolor=low_color, alpha = .5)

    plt.fill_between(merged_df.index, merged_df['atm7'], 0, 
                     where=(slippage >= high_quantile),
                     facecolor=high_color, alpha = .5)
    
    plt.plot(merged_df.index, merged_df['atm7'], color='black', linewidth=1)

    ax.set_ylim(merged_df['atm7'].min()*.975, merged_df['atm7'].max()*1.025)

    plt.title(title)
    plt.ylabel('ETH ATM7 Implied Volatility')

    low_legend = mpatches.Patch(color=low_color, label= '≤ 25th Percentile')
    high_legend = mpatches.Patch(color=high_color, label= '≥ 75th Percentile')
    plt.legend(handles=[low_legend, high_legend])

    plt.show()

In [ ]:
orderbook['slippage_median'] = orderbook['sell_percent_slippage'].rolling(60*24).mean()

In [ ]:
# Plotting the column 'buy/sell_dollar_slippage' against its index
plt.plot(orderbook.index, orderbook['slippage_median'], linewidth = 0.5, color = 'black', label='Sell Slippage')
# Adding labels to the x and y axes
plt.ylabel('Percent Slippage(%)')
plt.title("Intra-day Binance ETH/USDT $1m Sell Slipage Rolling Median")

# Rotate the x-axis labels by 45 degrees to prevent overlap
plt.xticks(rotation=45)

# Set the x-axis major tick format to display hours
plt.gcf().axes[0].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.gcf().set_dpi(150)
# Display the plot
plt.show()

In [ ]:
resampled_orderbook = orderbook.resample('4H').mean()

In [ ]:
# Plotting the column 'buy/sell_dollar_slippage' against its index
plt.plot(resampled_orderbook.index, resampled_orderbook['buy_percent_slippage'], linewidth = 0.2, color = 'red', label='Buy Slippage')
plt.plot(resampled_orderbook.index, resampled_orderbook['sell_percent_slippage'], linewidth = 0.2, color = 'green', label='Sell Slippage')
# Adding labels to the x and y axes
plt.ylabel('Percent Slippage(%)')
plt.title("Intra-day Binance ETH/USDT $1m Buy & Sell Order Slippage")

# Rotate the x-axis labels by 45 degrees to prevent overlap
plt.xticks(rotation=45)

# Set the x-axis major tick format to display hours
plt.gcf().axes[0].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.gcf().set_dpi(150)
# Display the plot
plt.show()

In [ ]:
orderbook['Slippage Difference'] = orderbook['sell_percent_slippage'] - orderbook['buy_percent_slippage']
# Plotting the column 'buy/sell_dollar_slippage' against its index
plt.plot(orderbook.index, orderbook['Slippage Difference'], linewidth = 0.5, color = 'black')

# Adding a faint grey dotted line at the 0 y axis mark
plt.axhline(y=0, linewidth = 0.5, color='black', linestyle='--', alpha=0.5)

# Adding labels to the x and y axes
plt.ylabel('Slippage Difference(%)')
plt.title("Intra-day Binance ETH/USDT $1m Sell-to-Buy Slippage Difference")

# Rotate the x-axis labels by 45 degrees to prevent overlap
plt.xticks(rotation=45)

# Set the x-axis major tick format to display hours
plt.gcf().axes[0].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.gcf().set_dpi(150)
# Display the plot
plt.show()

In [ ]:
create_heatmap(orderbook, 'sell_percent_slippage', '$1,000,000 ETH/USDT Sell Order Slippage % Heatmap', 'Reds')
create_heatmap(orderbook, 'buy_percent_slippage', '$1,000,000 ETH/USDT Buy Order Slippage % Heatmap', 'Greens')

In [ ]:
API_Key = "INSERT_YOUR_API_KEY_HERER"

def get_data(start_date, end_date, headers):
    url = f"https://web3api.io/api/v2/market/spot/ohlcv/eth_usd/historical?exchange=bitstamp&startDate={start_date}&endDate={end_date}&timeInterval=hours"
    response = requests.get(url, headers=headers)
    data = json.loads(response.text)
    ohlc = pd.DataFrame(data['payload']['data']['bitstamp'], columns=data['payload']['metadata']['columns'])
    ohlc['timestamp'] = pd.to_datetime(ohlc['timestamp'], unit='ms')
    ohlc.set_index('timestamp', inplace=True)
    ohlc.columns = [col.title() for col in ohlc.columns]
    return ohlc

headers = {
    "accept": "application/json",
    "x-api-key": API_Key
}

ohlc1 = get_data("2022-01-01T00%3A00%3A00", "2022-02-01T00%3A00%3A00", headers)
ohlc2 = get_data("2022-02-01T00%3A00%3A00", "2022-03-01T00%3A00%3A00", headers)
ohlc3 = get_data("2022-03-01T00%3A00%3A00", "2022-04-01T00%3A00%3A00", headers)
ohlc4 = get_data("2022-04-01T00%3A00%3A00", "2022-05-01T00%3A00%3A00", headers)
ohlc5 = get_data("2022-05-01T00%3A00%3A00", "2022-06-01T00%3A00%3A00", headers)
ohlc6 = get_data("2022-06-01T00%3A00%3A00", "2022-07-01T00%3A00%3A00", headers)
ohlc7 = get_data("2022-07-01T00%3A00%3A00", "2022-08-01T00%3A00%3A00", headers)
ohlc8 = get_data("2022-08-01T00%3A00%3A00", "2022-09-01T00%3A00%3A00", headers)
ohlc9 = get_data("2022-09-01T00%3A00%3A00", "2022-10-01T00%3A00%3A00", headers)
ohlc10 = get_data("2022-10-01T00%3A00%3A00", "2022-11-01T00%3A00%3A00", headers)
ohlc11 = get_data("2022-11-01T00%3A00%3A00", "2022-12-01T00%3A00%3A00", headers)
ohlc12 = get_data("2022-12-01T00%3A00%3A00", "2023-01-01T00%3A00%3A00", headers)
ohlc13 = get_data("2023-01-01T00%3A00%3A00", "2023-02-01T00%3A00%3A00", headers)
ohlc14 = get_data("2023-02-01T00%3A00%3A00", "2023-03-01T00%3A00%3A00", headers)

df = pd.concat([ohlc1, ohlc2, ohlc3, ohlc4, ohlc5, ohlc6, ohlc7, ohlc8, ohlc9, ohlc10, ohlc11, ohlc12, ohlc13, ohlc14], axis=0, join='inner', ignore_index=False)

In [ ]:
def parksinson_volatility(prices, window=24, trading_periods=365, clean=True):

    rs = (1.0 / (4.0 * math.log(2.0))) * ((prices['High'] / prices['Low']).apply(np.log))**2.0

    def f(v):
        return (trading_periods * v.mean())**0.5
    
    result = rs.rolling(
        window=window,
        center=False
    ).apply(func=f)
    
    if clean:
        return result.dropna()
    else:
        return result

df['hv'] = parksinson_volatility(df)

In [ ]:
# Plotting the column 'buy/sell_dollar_slippage' against its index
plt.plot(df.index, df['hv'], linewidth = 0.5, color = 'black', label='Sell Slippage')
# Adding labels to the x and y axes
plt.ylabel('Realized Volatility')
plt.title("Bitstamp ETH/USDT 24h Realized Vol (Parkinson)")

# Rotate the x-axis labels by 45 degrees to prevent overlap
plt.xticks(rotation=45)

# Set the x-axis major tick format to display hours
plt.gcf().axes[0].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.gcf().set_dpi(100)
# Display the plot
plt.show()

In [ ]:
# Plotting the column 'buy/sell_dollar_slippage' against its index
plt.plot(df.index, df['Volume'], linewidth = 0.5, color = 'black', label='Sell Slippage')
# Adding labels to the x and y axes
plt.ylabel('Realized Volatility')
plt.title("Intra-day Bitstamp 1hr ETH/USDT Volume")

# Rotate the x-axis labels by 45 degrees to prevent overlap
plt.xticks(rotation=45)

# Set the x-axis major tick format to display hours
plt.gcf().axes[0].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))
plt.gcf().set_dpi(100)
# Display the plot
plt.show()

In [ ]:
orderbook = orderbook.resample('H').mean()

In [ ]:
merged_df = df.merge(orderbook, left_index=True, right_index=True)

In [ ]:

url = "https://app.pinkswantrading.com/graphql"
headers = {
          'x-oracle': str(API_Key),
          'Content-Type': 'application/json',
          'accept': '*/*',
          'Accept-Language': 'en-US,en;q=0.9'
        }

payload="{\"query\":\"query ConstantMaturityAtm1Min($symbol: BTCOrETHEnumType, $dateStart: String, $dateEnd: String, $interval: String){\\n  ConstantMaturityAtm1Min(symbol:$symbol, dateStart:$dateStart, dateEnd: $dateEnd, interval: $interval) {\\n    date\\n    atm7\\n    atm30\\n    atm60\\n    atm90\\n    atm180\\n  }\\n}\\n\",\"variables\":{\"symbol\":\"ETH\",\"dateStart\":\"2022-01-01\",\"dateEnd\":\"2023-02-08\",\"interval\":\"(1 hour)\"}}"
response = requests.request("GET", url, headers=headers, data=payload)

iv = response.json()
iv = pd.DataFrame(iv['data']['ConstantMaturityAtm1Min'])
iv['timestamp'] = pd.to_datetime(iv['date'], unit='ms')
iv = iv.sort_values(by='timestamp', ascending=True)
iv = iv.set_index('timestamp')
iv = iv.drop(['date'], axis = 1)
iv

In [ ]:
merged_df = merged_df.merge(iv, left_index=True, right_index=True)

In [ ]:
create_heatmap(merged_df, 'atm7', 'Deribit ETH/USD ATM7 Implied Volatility Heatmap', 'Oranges')
create_heatmap(merged_df, 'hv', 'Bitstamp ETH/USDT Realized Volatility Heatmap', 'Blues')
create_heatmap(merged_df, 'Volume', 'Bitstamp ETH/USDT Volume Heatmap', 'Purples')

In [ ]:
correlation = merged_df['buy_percent_slippage'].corr(merged_df['hv'])
correlation

In [ ]:
x = merged_df['Volume']
y = merged_df['sell_percent_slippage']
xlabel = "Volume"
ylabel = "$1m Sell Order Slippage"
title = "Regression Plot between $1m Sell Order Slippage (%) and Volume"
color = 'black'
size = 2.5
alpha = 1

plot_regression(x, y, xlabel, ylabel, title, color, size, alpha)

In [ ]:
x = merged_df['Volume']
y = merged_df['buy_percent_slippage']
xlabel = "Volume"
ylabel = "$1m Buy Order Slippage"
title = "Regression Plot between $1m Buy Order Slippage (%) and Volume"
color = 'black'
size = 2.5
alpha = 1

plot_regression(x, y, xlabel, ylabel, title, color, size, alpha)

In [ ]:
x = merged_df['atm7']
y = merged_df['buy_percent_slippage']
xlabel = "ATM7 Implied Volatility"
ylabel = "$1m Buy Order Slippage"
title = "Regression Plot between $1m Buy Order Slippage (%) and ATM7 Implied Volatility"
color = 'black'
size = 2.5
alpha = 1

plot_regression(x, y, xlabel, ylabel, title, color, size, alpha)

In [ ]:
# Call the function for buy slippage
slippage_buy = merged_df['buy_percent_slippage']
plot_slippage(slippage_buy, 'ETH/USDT 1hr Close Price vs. $1m Buy Order Slippage')

# Call the function for sell slippage
slippage_sell = merged_df['sell_percent_slippage']
plot_slippage(slippage_sell, 'ETH/USDT 1hr Close Price vs. $1m Sell Order Slippage')

In [ ]:
# Call the function for buy slippage
slippage_buy = merged_df['hv']
plot_slippage(slippage_buy, 'ETH/USDT 1hr Close Price vs. Hist. Volatility (C/C)')

In [ ]:
# Call the function for buy slippage
slippage_buy = merged_df['atm30']
plot_slippage(slippage_buy, 'ETH/USDT 1hr Close Price vs. ATM 30 Implied Volatility')

# Call the function for sell slippage
slippage_sell = merged_df['atm30']
plot_slippage(slippage_sell, 'ETH/USDT 1hr Close Price vs. ATM 30 Implied Volatility')

In [ ]:
# Call the function for buy slippage
slippage_buy = merged_df['buy_percent_slippage']
plot_slippage_hv(slippage_buy, 'ETH/USDT 24hr Hist. Volatility (Parksinson) vs. $1m Buy Order Slippage')

# Call the function for sell slippage
slippage_sell = merged_df['sell_percent_slippage']
plot_slippage_hv(slippage_sell, 'ETH/USDT 24hr Hist. Volatility (Parksinson) vs. $1m Sell Order Slippage')

In [ ]:
# Call the function for buy slippage
slippage_buy = merged_df['buy_percent_slippage']
plot_slippage_iv(slippage_buy, 'ETH ATM7 Implied Volatility vs. $1m Buy Order Slippage')

# Call the function for sell slippage
slippage_sell = merged_df['sell_percent_slippage']
plot_slippage_iv(slippage_sell, 'ETH ATM7 Implied Volatility vs. $1m Sell Order Slippage')